<b>1.Data ETL</b>

<b>a. First, filter out all rows where the sentiment (‘context’) is not in the list of
aforementioned list of sentiments i.e. {'sad', 'jealous', 'joyful', 'terrified'}. </b><br>
In order to achieve this we will first reach the train and valid csv file and read it to the dataframe. Post this we will identify the rows where the context is <i>'sad', 'jealous', 'joyful', 'terrified'</i> and put it in a new dataframe. Post this we will merege the train and valid dataframes into a train_data dataframe<br>

<b>b. Synthesize your training attributes and label</b><br>
To achieve this we will extract only the Utterance and Context from the train_data and pass it as Attribute and Label to a new dataframe called final_train_dataset

In [1]:
'''
--------------------------------------------
Performing Data ETL
--------------------------------------------
'''
import pandas as pd

sentiment_list = {'sad', 'jealous', 'joyful', 'terrified'}

#Read the csv files using the relative path
train_file = r'C:\Users\Sanjana\Downloads\empatheticdialogues\empatheticdialogues\train.csv'
valid_file = r'C:\Users\Sanjana\Downloads\empatheticdialogues\empatheticdialogues\valid.csv'
test_file = r'C:\Users\Sanjana\Downloads\empatheticdialogues\empatheticdialogues\test.csv' 

#read the csv files and store them in the dataframe also exclude the errored/bad lines
df_train = pd.read_csv(train_file, error_bad_lines=False)
df_valid = pd.read_csv(valid_file, error_bad_lines=False)
df_test = pd.read_csv(test_file, error_bad_lines=False)

b'Skipping line 2355: expected 8 fields, saw 10\nSkipping line 36628: expected 8 fields, saw 12\nSkipping line 49433: expected 8 fields, saw 10\nSkipping line 56957: expected 8 fields, saw 10\nSkipping line 65019: expected 8 fields, saw 10\n'
b'Skipping line 4342: expected 8 fields, saw 10\n'
b'Skipping line 2168: expected 8 fields, saw 10\n'


In [2]:
#Segregate data where context is 'sad', 'jealous', 'joyful', 'terrified'
train_data = df_train[(df_train.context == 'sad')|(df_train.context == 'jealous')|
                      (df_train.context == 'joyful')|(df_train.context == 'terrified')]

#Get Test data
test_data = df_test[(df_test.context == 'sad')|(df_test.context == 'jealous')|
                      (df_test.context == 'joyful')|(df_test.context == 'terrified')]

#Merge the test and validation dataset
train_data = train_data.append(df_valid[(df_valid.context == 'sad')|(df_valid.context == 'jealous')|
                      (df_valid.context == 'joyful')|(df_valid.context == 'terrified')])

In [3]:
'''
--------------------------------------------
Synthesize the attribute and labels
--------------------------------------------
'''
#Train Dataset
final_train_dataset = pd.DataFrame()
final_train_dataset['Attribute'] = train_data['utterance']
final_train_dataset['Label'] = train_data['context']

#Test Dataset
final_test_dataset = pd.DataFrame()
final_test_dataset['Attribute'] = test_data['utterance']
final_test_dataset['Label'] = test_data['context']

In [4]:
print('Train dataset shape: '+str(final_train_dataset.shape))
print('Test dataset shape: '+str(final_test_dataset.shape))

Train dataset shape: (10939, 2)
Test dataset shape: (1349, 2)


<b>2. Convert the utterances into a sparse bag-of-words (BOW) </b><br>

In order to achieve this
1. Take the attribute column from final_train_dataset.Attribute and convert it to a sentence array
2. Ensure to lowercase it for easy comparision
3. Use the sklearn.CountVectorizer to count the occurances of words and form a matriz of features 

In [5]:
'''
--------------------------------------------
Convert rows in dataframes to lower and add 
them to a sentence array
--------------------------------------------
'''
word_arr = []
for i in final_train_dataset.Attribute:
    word_arr.append(i.lower())
    
#Sample
word_arr[:15]

['job interviews always make me sweat bullets_comma_ makes me uncomfortable in general to be looked at under a microscope like that. ',
 "don't be nervous. just be prepared.",
 'i feel like getting prepared and then having a curve ball thrown at you throws you off. ',
 'yes but if you stay calm it will be ok.',
 "it's hard to stay clam. how do you do it? ",
 'hi_comma_ this year_comma_ i was the first over 300 students at my enginering school',
 "sounds great! so what's your major?",
 'it is computer science. i am very happy of this achievement and my family is very proud.',
 "well pleased. you should be having brains_comma_man!that's a tough course_comma_i hear.",
 'during christmas a few years ago_comma_ i did not get any presents.',
 'wow_comma_ that must be terrible_comma_ i cannot imagine_comma_ i lvoe christmas',
 'since that day christmas has not been a good time for me. as i have no family_comma_ christmas is always the worst.',
 'wow_comma_ i am sorry to hear that_comma_ i wis

In [6]:
'''
--------------------------------------------
Count Vectorizer to find B-O-W
--------------------------------------------
'''
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

#Count of the Occurances of the word
count_occurs = vectorizer.fit_transform(word_arr)

#Get the features/words
features = vectorizer.get_feature_names()
#print(features)
feature_matrix = pd.DataFrame(count_occurs.toarray(),columns = features)

#Sample
feature_matrix.head()

,00am,10,100,1000,100_comma_000,100f,100k,100x,10_comma_,10_comma_000,...,zelda,zelda_comma_,zero,zimbabwe,zip,ziplining,zone,zoo,zoom,zoom_comma_
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#Sample occurances for the word christmas in each statement
feature_matrix.christmas.head(15)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     1
10    1
11    2
12    0
13    0
14    0
Name: christmas, dtype: int64

In [8]:
'''
--------------------------------------------
Remove Stopwords from the sentences
--------------------------------------------
'''

from nltk.corpus import stopwords
import string

stop_words = stopwords.words("english")

#Remove Punctuation
punct = list(string.punctuation)

#Remove other unecessary words
other_words = ['comma', 'am', 'pm','ugh', 'aagh', 'jeez','woah', 'oh', 'ooh', 'aall', 'ab', 'hmmm', 'zoo', 'zoos',
               'you', 'yous','youve', 'yourself', 'zimbabwe', 'zealand','yursef','zip','zoom','ziplining', 'yyouca']
for i in punct:
    stop_words.append(i)
for i in other_words:
    stop_words.append(i)

#Remove digits
for i in range(len(word_arr)):
    word_arr[i] = ''.join(c for c in word_arr[i] if not c.isdigit())
for i in range(len(word_arr)):
    word_arr[i] = ''.join(c for c in word_arr[i] if not c.isdigit())

In [9]:
'''
--------------------------------------------
Refine the words and genrated sentences
--------------------------------------------
'''
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import re

#Use wordNet Lemmatizer to lemmatize the words and find the stem words
lemmatizer = WordNetLemmatizer()

word_arr_no_stop_train = []
for sentence in word_arr:
    tokens = nltk.word_tokenize(sentence)
    sent_no_stop = ""
    i = 0
    for word in tokens:
        i += 1
        word = lemmatizer.lemmatize(word,'v')#Verb
        word = lemmatizer.lemmatize(word,'n')#Noun
        word = lemmatizer.lemmatize(word,'a')#Adjective
        word = re.sub('[^a-z\s]'," ",word)
        if word not in stop_words:
            if i==1:
                sent_no_stop +=''.join(word)
            else:
                sent_no_stop +=" "+''.join(word)
    word_arr_no_stop_train.append(sent_no_stop)  

In [10]:
#Remove extra spaces
for i in range(len(word_arr_no_stop_train)):
    word_arr_no_stop_train[i] = " ".join(word_arr_no_stop_train[i].split())

In [11]:
#Remove words with one letter
for i in range(len(word_arr_no_stop_train)):
    word_arr_no_stop_train[i] = " ".join(w for w in word_arr_no_stop_train[i].split() if len(w)>1)

In [12]:
#Remove the word 'comma'
for i in range(len(word_arr_no_stop_train)):
    word_arr_no_stop_train[i] = " ".join(w for w in word_arr_no_stop_train[i].split() if w!='comma')

In [13]:
'''
--------------------------------------------
Count Vectorizer to find B-O-W
--------------------------------------------
'''
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
count_occurs = vectorizer.fit_transform(word_arr_no_stop_train)

features = vectorizer.get_feature_names()

#print(features)
feature_matrix = pd.DataFrame(count_occurs.toarray(),columns = features)

#Columns Sample
feature_matrix.columns

Index(['abandon', 'abduction', 'abilities', 'ability', 'able', 'abnout',
       'abortion', 'about', 'abouts', 'abroad',
       ...
       'yuck', 'yukon', 'yum', 'yummy', 'yup', 'zelda', 'zero', 'zip', 'zone',
       'zoom'],
      dtype='object', length=5759)

In [14]:
#Sample
feature_matrix.christmas.head(15)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     1
10    1
11    2
12    0
13    0
14    0
Name: christmas, dtype: int64

<b>4. Normalization</b><br>

In order to achieve this we use TfIDF. In thi case we are trying bothe TDIDFVEctorizer and TFIDFTransfomer.

In [15]:
'''
--------------------------------------------
Tfidf Vectorizer normalie the occurances
--------------------------------------------
'''
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vactorizer = TfidfVectorizer()
tfidf_counts = tfidf_vactorizer.fit_transform(word_arr_no_stop_train)

features = tfidf_vactorizer.get_feature_names()
#print(features)
feature_matrix = pd.DataFrame(tfidf_counts.toarray(),columns = features)

#Print the normalized sample
feature_matrix.head()

,abandon,abduction,abilities,ability,able,abnout,abortion,about,abouts,abroad,...,yuck,yukon,yum,yummy,yup,zelda,zero,zip,zone,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Normalized occurance count for the word christmas
feature_matrix.christmas.head(15)

0     0.000000
1     0.000000
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.000000
7     0.000000
8     0.000000
9     0.520043
10    0.417727
11    0.740580
12    0.000000
13    0.000000
14    0.000000
Name: christmas, dtype: float64

In [17]:
final_train_dataset['Train'] = word_arr_no_stop_train
final_train_dataset.head(5)

,Attribute,Label,Train
21,Job interviews always make me sweat bullets_co...,terrified,job interview always make sweat bullets make u...
22,Don't be nervous. Just be prepared.,terrified,nervous prepare
23,I feel like getting prepared and then having a...,terrified,feel like get prepare curve ball throw throw
24,Yes but if you stay calm it will be ok.,terrified,yes stay calm ok
25,It's hard to stay clam. How do you do it?,terrified,hard stay clam


<b> Repeat the above steps for the test dataset </b>

In [18]:
'''
--------------------------------------------
Convert rows in dataframes to lower and add 
them to a sentence array
--------------------------------------------
'''
word_arr = []
for i in final_test_dataset.Attribute:
    word_arr.append(i.lower())

In [19]:
'''
--------------------------------------------
Remove Stopwords from the sentences
--------------------------------------------
'''
from nltk.corpus import stopwords
import string

stop_words = stopwords.words("english")
punct = list(string.punctuation)
other_words = ['comma', 'am', 'pm', 'ugh', 'aagh', 'jeez','woah', 'oh', 'ooh', 'aall', 'ab', 'hmmm', 'zoo', 'zoos'
                'you', 'yous','youve', 'yourself','zimbabwe', 'zealand','yursef','zip','zoom','ziplining', 'yyouca']
for i in punct:
    stop_words.append(i)
for i in other_words:
    stop_words.append(i)
    
for i in range(len(word_arr)):
    word_arr[i] = ''.join(c for c in word_arr[i] if not c.isdigit())
for i in range(len(word_arr)):
    word_arr[i] = ''.join(c for c in word_arr[i] if not c.isdigit())

In [20]:
'''
--------------------------------------------
Refine the words and generated sentences
--------------------------------------------
'''
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
import re

word_arr_no_stop = []
lemmatizer = WordNetLemmatizer()
word_arr_no_stop = []
for sentence in word_arr:
    tokens = nltk.word_tokenize(sentence)
    sent_no_stop = ""
    i = 0
    for word in tokens:
        i += 1
        word = lemmatizer.lemmatize(word,'v')
        word = lemmatizer.lemmatize(word,'n')
        word = lemmatizer.lemmatize(word,'a')
        word = re.sub('[^a-z\s]'," ",word)
        if word not in stop_words:
            if i==1:
                sent_no_stop +=''.join(word)
            else:
                sent_no_stop +=" "+''.join(word)
    word_arr_no_stop.append(sent_no_stop) 

In [21]:
# Remove extra spaces
for i in range(len(word_arr_no_stop)):
    word_arr_no_stop[i] = " ".join(word_arr_no_stop[i].split())

In [22]:
#Remove single letter words
for i in range(len(word_arr_no_stop)):
    word_arr_no_stop[i] = " ".join(w for w in word_arr_no_stop[i].split() if len(w)>1)

In [23]:
# Remove the word 'comma'
for i in range(len(word_arr_no_stop)):
    word_arr_no_stop[i] = " ".join(w for w in word_arr_no_stop[i].split() if w!='comma')

In [24]:
'''
--------------------------------------------
Count Vectorizer to find B-O-W
--------------------------------------------
'''
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
count_occurs = vectorizer.fit_transform(word_arr_no_stop)

features = vectorizer.get_feature_names()
#print(features)
feature_matrix = pd.DataFrame(count_occurs.toarray(),columns = features)

#Columns Sample
feature_matrix.columns

Index(['abandon', 'abilities', 'ability', 'able', 'absolutely', 'accent',
       'accept', 'acceptance', 'accomplish', 'accomplishment',
       ...
       'yes', 'yesterday', 'yet', 'yikes', 'you', 'young', 'younger',
       'yourself', 'youtube', 'zoos'],
      dtype='object', length=2030)

In [25]:
'''
--------------------------------------------
Tfidf Vectorizer normalize the occurances
in the test dataset
--------------------------------------------
'''
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vactorizer = TfidfVectorizer()
tfidf_counts = tfidf_vactorizer.fit_transform(word_arr_no_stop)

features = tfidf_vactorizer.get_feature_names()

#print(features)
feature_matrix = pd.DataFrame(tfidf_counts.toarray(),columns = features)

#Print the normalized sample
feature_matrix.head()

,abandon,abilities,ability,able,absolutely,accent,accept,acceptance,accomplish,accomplishment,...,yes,yesterday,yet,yikes,you,young,younger,yourself,youtube,zoos
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.181151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
final_test_dataset['Test'] = word_arr_no_stop
final_test_dataset.head(5)

,Attribute,Label,Test
17,I'm so sad because i've read an article about ...,sad,sad ve read article newborn girl die parent be...
18,Ugh_comma_ those articles always get me too......,sad,ugh article always get wrong
19,she was born premature at home_comma_ she had ...,sad,bear premature home hard time breathe instead ...
20,Jeez! Its so unfortunate... very sad really.,sad,unfortunate sad really
21,yes! And i do believe in God and prayers but g...,sad,yes believe god prayer goodness gracious pleas...


<b>5. SGDClassifier</b><br>

In order to achieve this we use TfIDF. In this case TfidfTransformer helps standardize the train and test dataset helping to fit the model and predict the labes for test dataset

As we can see below we get and accuracy of <b>62.4%</b>

In [27]:
'''
--------------------------------------------
Using SGD Classifier
--------------------------------------------
'''
from sklearn.linear_model import SGDClassifier
import numpy as np

#Defining the train and test attributes and labels
trainDocs,trainLabels = word_arr_no_stop_train, final_train_dataset.Label
testDocs,testLabels = word_arr_no_stop, test_data.context

#Use CountVectorizer and TfidfTransformer on the train dataset
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(trainDocs)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_train_array = X_train_tfidf.toarray()

#Use CountVectorizer and TfidfTransformer on the test dataset
X_test_counts = count_vect.transform(testDocs)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)


X_test_array = X_test_tfidf.toarray()

#Define the classifier and add parameters like alpha, max_iter, penalty etc.
clf = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, tol=1e-3, max_iter=1000)

#Fit the model on the train dataset
clf.fit(X_train_tfidf,np.array(trainLabels))

#Predict the values for teh test dataset and compare the results
predicted = clf.predict(X_test_tfidf)
acc_tfidf = np.mean(predicted == np.array(testLabels))

In [28]:
#Below are the results
print('Accuracy of the model is:'+ str(round(acc_tfidf*100,2))+'%')

Accuracy of the model is:62.05%


#### Question 6 
1. Fine tuning the results means using the pretrained model to help us get better results by looking at the correlations to optimize the results. In our scenario we have the conversation chatbot. The average length of each conversation is close to 4. In this scenario we try to fine tune the results by checking the context window of the previous 4 utterances to make the model predict the context of the next utterance. This can help us improve the results by correlating to the context available making the prediction more accurate thus, helping the chatbot understand the context of the situation. However,finetuning needs to add weights to the previous occurances of the context to the most recent one
<br>
2. In a transformer normally each encoder is similar to one another and so are the decoders. Another property of transformers is that each word in the encoder has its own path. That is the processing and feed forward happens at a faster pace i.e. there are no dependencies thus, increasing the pace. Also encoder and decoder decide the weights of a word depending on the other words in the sentence. Correlating the words helps in undertsanding the context. These properties help transformer to provide better results for the empathetic chatbot
<br>
3. P@1,100: This is the accuracy of the model of choosing a correct response out of the given 100 randomly selected test set. <br>
    AVG-BLEU: BLEU score is used to evaluate a generated senetence to a reference sentence. BLEU uses the computation of precision using n-gram for this and depending on the type of n-gram used we have BLEU-1-2-3-4. In this study we are using BLEU which also means the average BLEU for the results from various BLEU approaches<br>
    PPL: Perplexity - Perplexity is an exponentiation of entropy. This captures the uncertainity in the model. Higher the entropy higher the perplexity and vice-versa
<br>
4. BLEU seems to the best metric as it used the correration of words to generate the meteric. As we can see BLEU-n uses the computation precision of the n-gram thus helping in correlating the words that often fall together. This is really important in the empathetic systems to undertstand the context
<br>
5. Currently the model is empathetic but not as empathetic as humans. To improve this we need to feed the model with more training dataset that can help in incorporating the same in general dialog. The improvements can be done with data and capacity. In case we have better. We can also improve the tuning model to take ito account the similarities in a context and also using the human trained pretrained dataset to help identify and fine tune it more

In [29]:
import multiprocessing
import sys
import gensim.models
from gensim.models.word2vec import Word2Vec

workers = multiprocessing.cpu_count()
print('number of cpu: {}'.format(workers))
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise."

number of cpu: 8


In [30]:
from nltk.tokenize import word_tokenize
tokenized_sents = [word_tokenize(i) for i in final_train_dataset['Train']]
print(len(tokenized_sents))

10939


In [31]:
word_model = Word2Vec(tokenized_sents,
                      min_count=2,
                      size=100,
                      window=5,
                      workers=workers,
                      iter=100)

In [32]:
from collections import defaultdict
class TfidfEmbeddingVectorizer(object):

    def __init__(self, word_model):

        self.word_model = word_model
        self.word_idf_weight = None
        self.vector_size = word_model.wv.vector_size

    def fit(self, docs):  # comply with scikit-learn transformer requirement
        """
        Fit in a list of docs, which had been preprocessed and tokenized,
        such as word bi-grammed, stop-words removed, lemmatized, part of speech filtered.
        Then build up a tfidf model to compute each word's idf as its weight.
        Noted that tf weight is already involved when constructing average word vectors, and thus omitted.
        :param
            pre_processed_docs: list of docs, which are tokenized
        :return:
            self
        """

        text_docs = []
        for doc in docs:
            text_docs.append(" ".join(doc))

        tfidf = TfidfVectorizer()
        tfidf.fit(text_docs)  # must be list of text string

        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of
        # known idf's
        max_idf = max(tfidf.idf_)  # used as default value for defaultdict
        self.word_idf_weight = defaultdict(lambda: max_idf,
                                               [(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()])
        return self


    def transform(self, docs):  # comply with scikit-learn transformer requirement
        doc_word_vector = self.word_average_list(docs)
        return doc_word_vector


    def word_average(self, sent):
        """
        Compute average word vector for a single doc/sentence.
        :param sent: list of sentence tokens
        :return:
            mean: float of averaging word vectors
        """

        mean = []
        for word in sent:
            if word in self.word_model.wv.vocab:
                mean.append(self.word_model.wv.get_vector(word) * self.word_idf_weight[word])  # idf weighted

        if not mean:  # empty words
            # If a text is empty, return a vector of zeros.
            #logging.warning("cannot compute average owing to no vector for {}".format(sent))
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean


    def word_average_list(self, docs):
        """
        Compute average word vector for multiple docs, where docs had been tokenized.
        :param docs: list of sentence in list of separated tokens
        :return:
            array of average word vector in shape (len(docs),)
        """
        return np.vstack([self.word_average(sent) for sent in docs])


In [33]:
tfidf_vec_tr  = TfidfEmbeddingVectorizer(word_model)
  # fit tfidf model first

tfidf_vec_tr.fit(tokenized_sents) 
tfidf_doc_vec = tfidf_vec_tr.transform(tokenized_sents)

# print('Demo of word averaging doc vector...')
# display(doc_vec[4])

In [34]:
# Save word averaging doc2vec.
print('Shape of tfidf-word-mean doc2vec...')
display(tfidf_doc_vec.shape)

tfidf_doc_vec

Shape of tfidf-word-mean doc2vec...


(10939, 100)

array([[  0.07985463,   3.65502405,  -0.54506034, ...,  -4.94287586,
          2.27880907,  -1.65137064],
       [ -8.77450943,   0.79331565,   4.41261625, ...,  -2.16055083,
        -10.44566917,   5.73236275],
       [-10.17860603,   0.05264745,   4.33068275, ...,  -2.35055733,
         -6.40414047,  -0.86498821],
       ...,
       [  4.95237494,  -9.69329166,   1.46142483, ...,  -0.03043954,
         -2.97034621,  -3.80776119],
       [  4.03734732,  -0.87526542,  -1.5348947 , ...,  -2.48545265,
          1.6004467 ,   3.0723021 ],
       [  0.15700443,  -3.01226926,  -1.27088082, ...,   0.26645681,
          3.87873197,   2.34608459]])

In [35]:
from nltk.tokenize import word_tokenize
tokenized_test_sents = [word_tokenize(i) for i in final_test_dataset['Test']]
print(len(tokenized_test_sents))

1349


In [36]:
tfidf_doc_vec_test = tfidf_vec_tr.transform(tokenized_test_sents)

In [37]:
# Save word averaging doc2vec.
print('Shape of tfidf-word-mean doc2vec...')
display(tfidf_doc_vec_test.shape)

tfidf_doc_vec_test

Shape of tfidf-word-mean doc2vec...


(1349, 100)

array([[-2.59118581,  0.75729626, -0.7386176 , ...,  2.6746769 ,
        -3.42214346,  5.54761124],
       [-0.26679391,  1.57986748, -2.63601422, ..., -3.4692893 ,
        -3.5446744 , -0.02520895],
       [-1.78760719,  2.11584401,  0.79802573, ...,  5.86673737,
        -1.70027959,  4.44860125],
       ...,
       [-3.13985419, -0.07907978, -1.02455497, ...,  0.30550432,
        -3.24897647, -3.78489423],
       [ 3.96508527,  5.43052292,  2.07830215, ..., -4.03631878,
         4.05159283,  4.79649544],
       [-1.52722943, -0.51442164, -1.34763908, ..., -2.00467086,
         3.65751672, -2.6253159 ]])

In [38]:
from sklearn.linear_model import SGDClassifier

# (Optional) Classification via stochastic gradient descent classifier.
sgd = SGDClassifier(loss='hinge',
                    penalty='l2',
                    verbose=1,
                    random_state=1,
                    learning_rate='invscaling',
                    eta0=1, alpha=0.0001, tol=1e-7, max_iter=10000)

sgd.fit(tfidf_doc_vec,final_train_dataset.Label)

-- Epoch 1
Norm: 3.19, NNZs: 100, Bias: -0.823357, T: 10939, Avg. loss: 6.883600
Total training time: 0.01 seconds.
-- Epoch 2
Norm: 1.94, NNZs: 100, Bias: -1.007264, T: 21878, Avg. loss: 1.895180
Total training time: 0.01 seconds.
-- Epoch 3
Norm: 1.86, NNZs: 100, Bias: -1.020977, T: 32817, Avg. loss: 1.519462
Total training time: 0.02 seconds.
-- Epoch 4
Norm: 1.64, NNZs: 100, Bias: -1.046844, T: 43756, Avg. loss: 1.355339
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 1.42, NNZs: 100, Bias: -1.013220, T: 54695, Avg. loss: 1.255594
Total training time: 0.04 seconds.
-- Epoch 6
Norm: 1.48, NNZs: 100, Bias: -1.046863, T: 65634, Avg. loss: 1.153463
Total training time: 0.04 seconds.
-- Epoch 7
Norm: 1.30, NNZs: 100, Bias: -1.063595, T: 76573, Avg. loss: 1.110509
Total training time: 0.05 seconds.
-- Epoch 8
Norm: 1.21, NNZs: 100, Bias: -1.028193, T: 87512, Avg. loss: 1.022082
Total training time: 0.06 seconds.
-- Epoch 9
Norm: 1.10, NNZs: 100, Bias: -1.021690, T: 98451, Avg. loss: 

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.6s finished


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=1, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='hinge', max_iter=10000,
       n_iter=None, n_jobs=1, penalty='l2', power_t=0.5, random_state=1,
       shuffle=True, tol=1e-07, verbose=1, warm_start=False)

In [39]:
from sklearn.metrics import *

train_pred = sgd.predict(tfidf_doc_vec)
print('Training accuracy:'+str(accuracy_score(train_pred,final_train_dataset.Label)))
print(classification_report(train_pred,final_train_dataset.Label))

Training accuracy:0.5444739007221867
             precision    recall  f1-score   support

    jealous       0.49      0.50      0.50      2546
     joyful       0.53      0.54      0.54      2553
        sad       0.53      0.52      0.52      3000
  terrified       0.63      0.61      0.62      2840

avg / total       0.55      0.54      0.54     10939



In [40]:
test_pred = sgd.predict(tfidf_doc_vec_test)
print('Training accuracy:'+str(accuracy_score(test_pred,final_test_dataset.Label)))
print(classification_report(test_pred,final_test_dataset.Label))


Training accuracy:0.519644180874722
             precision    recall  f1-score   support

    jealous       0.41      0.47      0.44       301
     joyful       0.46      0.54      0.50       301
        sad       0.55      0.51      0.53       397
  terrified       0.68      0.56      0.61       350

avg / total       0.53      0.52      0.52      1349



In [41]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(alpha=0.01, hidden_layer_sizes=(64,128,512,512,1024,64,32), max_iter=1000, activation='relu', solver='adam')

#train_and_evaluate(mlp,X_train_w2v,final_train_dataset.Label,X_test_w2v,final_test_dataset.Label)
mlp.fit(tfidf_doc_vec,final_train_dataset.Label)

MLPClassifier(activation='relu', alpha=0.01, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(64, 128, 512, 512, 1024, 64, 32),
       learning_rate='constant', learning_rate_init=0.001, max_iter=1000,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [42]:
train_mlp_pred = mlp.predict(tfidf_doc_vec)
print('Training accuracy:'+str(accuracy_score(train_mlp_pred,final_train_dataset.Label)))
print(classification_report(train_mlp_pred,final_train_dataset.Label))

Training accuracy:0.9666331474540635
             precision    recall  f1-score   support

    jealous       0.96      0.98      0.97      2527
     joyful       0.98      0.94      0.96      2729
        sad       0.96      0.97      0.97      2945
  terrified       0.97      0.98      0.97      2738

avg / total       0.97      0.97      0.97     10939



In [43]:
test_mlp_pred = mlp.predict(tfidf_doc_vec_test)
print('Training accuracy:'+str(accuracy_score(test_mlp_pred,final_test_dataset.Label)))
print(classification_report(test_mlp_pred,final_test_dataset.Label))

Training accuracy:0.5189028910303929
             precision    recall  f1-score   support

    jealous       0.42      0.52      0.47       274
     joyful       0.53      0.53      0.53       346
        sad       0.52      0.49      0.51       393
  terrified       0.62      0.53      0.57       336

avg / total       0.53      0.52      0.52      1349

